# Joining Farecard Stations with Coordinates

Two datasets were downloaded, one that includes farecard usage data at every station, and one that includes the coordinates of stations. The goal is to join them so that we can compare farecard usage with other localized data, like traffic conditions or motor vehicle collisions.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# First dataset - farecard usage
farecard = pd.read_csv('MTA_FareCardUsage.csv')

# Second dataset - stations with coordinates
stations = pd.read_csv('stations.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,12,13,16,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
farecard.head()

,From_Date,To_Date,Remote_Station_ID,Station,Full_Fare,Senior_Citizen___Disabled,Seven_Day_ADA_Farecard_Access_System_Unlimited,Thirty_Day_ADA_Farecard_Access_System_Unlimited,Joint_Rail_Road_Ticket,Seven_Day_Unlimited,...,LIB_Special_Senior,Rail_Road_Unlimited_No_Trade,Transit_Check_Metrocard_Annual_Metrocard,Mail_and_Ride_EZPass_Express,Mail_and_Ride_Unlimited,Path_2_Trip,Airtran_Full_Fare,Airtran_Thirty_Day,Airtran_One0_Trip,Airtran_Monthly
0,1/19/19,1/25/19,R001,WHITEHALL STREET,"51,485","2,392",347,"1,003",219,"21,801",...,528,328,"5,448","2,012",735,0,"1,111",0,0,0
1,1/19/19,1/25/19,R003,CYPRESS HILLS,"2,657",143,12,34,0,"2,134",...,79,5,94,26,27,0,21,0,0,0
2,1/19/19,1/25/19,R004,75TH STREET & ELDERTS LANE,"6,390",354,57,138,2,"4,604",...,203,0,283,39,60,0,55,0,0,0
3,1/19/19,1/25/19,R005,85TH STREET & FOREST PKWAY,"7,066",497,71,178,1,"4,424",...,195,5,246,57,61,0,55,0,0,0
4,1/19/19,1/25/19,R006,WOODHAVEN BOULEVARD,"7,117",450,56,167,0,"6,444",...,233,11,338,70,65,0,61,0,0,0


In [4]:
stations.head()

,Unnamed: 0,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude
0,0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034
1,1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843
2,2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479
3,3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508
4,4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575


We can't easily join these datasets because the 'Remote_Station_ID' field from the first one doesn't correspond to 'GTFS Stop ID' from the second. But we also have a 3rd dataset that gives us the lines that stop at each Remote_Station_ID. We'll join the lines to the farecard dataset to get a list of station names and lines.

In [9]:
remote = pd.read_excel('Remote-Booth-Station.xls')
remote.head()

,Remote,Booth,Station,Line Name,Division
0,R001,A060,WHITEHALL ST,R1,BMT
1,R001,A058,WHITEHALL ST,R1,BMT
2,R001,R101S,SOUTH FERRY,R1,IRT
3,R002,A077,FULTON ST,ACJZ2345,BMT
4,R002,A081,FULTON ST,ACJZ2345,BMT


In [10]:
remote_lines = remote.groupby('Remote')['Line Name'].first().reset_index()
remote_lines.head()

,Remote,Line Name
0,R001,R1
1,R002,ACJZ2345
2,R003,J
3,R004,JZ
4,R005,J


In [11]:
fstations = farecard[['Remote_Station_ID', 'Station']].drop_duplicates().rename(index=str, columns={"Remote_Station_ID": 'Remote'})
fstations = fstations.merge(remote_lines, on='Remote', how='left')

In [12]:
fstations.head()

,Remote,Station,Line Name
0,R001,WHITEHALL STREET,R1
1,R003,CYPRESS HILLS,J
2,R004,75TH STREET & ELDERTS LANE,JZ
3,R005,85TH STREET & FOREST PKWAY,J
4,R006,WOODHAVEN BOULEVARD,JZ


Now, some preprocessing steps:
- separate numbers from ordinal endings
- remove punctuation
- add '_TRAIN' to train lines to make sure they're distinguishable from street numbers

In [5]:
def sepdigits(s):
    last = 'a'
    for i, c in enumerate(s):
        if last.isdigit() and not c.isdigit():
            return s[0:i] + ' ' + sepdigits(s[i:])
        last = c
    return s

In [6]:
def format(r):
    s = sepdigits(r[0].replace('-', ' ').replace('/', ' ').replace('AVENUE', 'Av'))
    if r[1] != 'nan':
        trains = [t + '_TRAIN' for t in list(r[1])]
        s = s + ' ' + ' '.join(trains)
    return s

In [13]:
fstations['Format_Name'] = [format(r) for r in zip(fstations.Station, fstations['Line Name'].astype(str))]

In [14]:
fstations.sample(5)

,Remote,Station,Line Name,Format_Name
462,R488,JACKIE GLEASON DEPOT(2),NaN,JACKIE GLEASON DEPOT(2 )
56,R060,EASTERN PARKWAY,23,EASTERN PARKWAY 2_TRAIN 3_TRAIN
206,R218,ELMHURST AVE-BROADWAY,MR,ELMHURST AVE BROADWAY M_TRAIN R_TRAIN
257,R269,BEDFORD AVENUE-NOSTRAND AVE,G,BEDFORD Av NOSTRAND AVE G_TRAIN
33,R037,207TH STREET,1,207 TH STREET 1_TRAIN


We'll also preprocess names from the 'stations' dataset in a similar way, merging the train lines with the station name and formatting the string.

In [15]:
def format_2(r):
    #s = r[0].replace('-', ' ')
    s = r[1].replace('-', ' ').replace('/', ' ') #s + ' ' + r[1].replace('-', ' ')
    trains = [t + '_TRAIN' for t in r[2].split(' ')]
    s = sepdigits(s) + ' ' + ' '.join(trains)
    return s

In [19]:
stations['Format_Name'] = [format_2(r) for r in zip(stations.Line, stations['Stop Name'], stations['Daytime Routes'])]

In [20]:
stations['Format_Name'].sample(10)

239                       Smith   9  Sts F_TRAIN G_TRAIN
388                                      170  St 4_TRAIN
395                                      110  St 6_TRAIN
458                                       52  St 7_TRAIN
118                                        1  Av L_TRAIN
437                             Harlem   148  St 3_TRAIN
60                                        50  St D_TRAIN
93                                     Halsey St J_TRAIN
160    59  St   Columbus Circle A_TRAIN B_TRAIN C_TRA...
403       Grand Central   42  St 4_TRAIN 5_TRAIN 6_TRAIN
Name: Format_Name, dtype: object

Now, we'll merge all the names from both datasets into a single 'corpus' of documents to do TF-IDF vectorization.

In [21]:
docs = stations['Format_Name'].append(fstations['Format_Name'])
docs.shape

(1505,)

In [22]:
vectorizer = TfidfVectorizer(token_pattern = u'(?u)\\b\\w+\\b')
X = vectorizer.fit(docs)

In [24]:
#TF-IDF representations of stations ('st') and farecard ('fc') datasets
st_tfidf = vectorizer.transform(stations['Format_Name'])
fc_tfidf = vectorizer.transform(fstations['Format_Name'])

Let's test how well the TF-IDF representations can be used to do string matching. We'll choose a random station from the farecard database, and print its nearest neighbor using cosine similarity.

In [27]:
k = np.random.randint(fstations.shape[0])
print("Entry from farecard dataset:")
print(k)
print(fstations.iloc[k])
dists = cosine_similarity(st_tfidf, fc_tfidf[k])[:,0]
i = np.argmax(dists)
print()
print("Nearest neighbor from stations dataset:")
print(stations.Name.iloc[i])

Entry from farecard dataset:
770
Remote                                            R260
Station                 181ST STREET-ST NICHOLAS AV   
Line Name                                            1
Format_Name    181 ST STREET ST NICHOLAS AV    1_TRAIN
Name: 770, dtype: object

Nearest neighbor from stations dataset:
181  St 1_TRAIN


In [28]:
ind = [np.argmax(cosine_similarity(st_tfidf, fc_tfidf[k])[:,0]) for k in range(fstations.shape[0])]

Based on looking at a few dozen entries, we've joined the names with higher accuracy than using any previous strategy.

In [32]:
fstations['Map_Station'] = stations.Name.iloc[ind].values
fstations['Latitude'] = stations['GTFS Latitude'].iloc[ind].values
fstations['Longitude'] = stations['GTFS Longitude'].iloc[ind].values

In [33]:
fstations.head()

,Remote,Station,Line Name,Format_Name,Map_Station,Latitude,Longitude
0,R001,WHITEHALL STREET,R1,WHITEHALL STREET R_TRAIN 1_TRAIN,Whitehall St R_TRAIN W_TRAIN,40.703087,-74.012994
1,R003,CYPRESS HILLS,J,CYPRESS HILLS J_TRAIN,Cypress Hills J_TRAIN,40.689941,-73.872550
2,R004,75TH STREET & ELDERTS LANE,JZ,75 TH STREET & ELDERTS LANE J_TRAIN Z_TRAIN,75 St J_TRAIN Z_TRAIN,40.691324,-73.867139
3,R005,85TH STREET & FOREST PKWAY,J,85 TH STREET & FOREST PKWAY J_TRAIN,85 St Forest Pkwy J_TRAIN,40.692435,-73.860010
4,R006,WOODHAVEN BOULEVARD,JZ,WOODHAVEN BOULEVARD J_TRAIN Z_TRAIN,Woodhaven Blvd J_TRAIN Z_TRAIN,40.693879,-73.851576


In [34]:
fstations.to_csv('Stations_2_Coords.csv', index=False)